In [1]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizerFast, BertForTokenClassification
from torch.utils.data import DataLoader
from torch.optim import SGD
from tqdm import tqdm

2023-06-28 15:43:41.382852: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-28 15:43:41.384376: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-28 15:43:41.414303: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-28 15:43:41.415176: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-28 15:43:42.026285: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
#read the dataset and rename columns
data = pd.read_csv("/home/shriya/Documents/NER/Transformer/dataset.csv")

data.rename(columns={"Sentence#":"sentence#","Sentence":"text" ,"POS":"POStags","Tag":"labels"}, inplace=True)
data.head(11)

,sentence#,text,POStags,labels
0,1,"' ಆವಿಷ್ಕಾರ , ಸುಸ್ಥಿರತೆ ಮತ್ತು ಬಹುಪಕ್ಷೀಯ ಸಹಕಾರ '...",RD_PUNC N_NN RD_PUNC N_NN CC_CCD JJ N_NN RD_PU...,O O O O O O O O O O NEL O NEL O O O NETE NETE O O
1,2,"ಮಾರ್ಚ್ 16 , 2022 .",N_NNP QT_QTC RD_PUNC QT_QTC RD_PUNC,NED NED O NEN O
2,3,"ಮಹಾಮಹಿಮ ಶ್ರೀ ಫ್ರಾಂಕೋಯಿಸ್ ಡೆಲಾಟ್ರೆ , ಪ್ರಧಾನ ಕಾರ...",N_NN N_NN N_NNP N_NNP RD_PUNC JJ N_NN RD_PUNC ...,O O NEP NEP O O O O NEL O NEL O O O O O
3,4,"ಮಹಾಮಹಿಮ ಶ್ರೀ ಇಮ್ಯಾನುಯೆಲ್ ಲೆನಿನ್ , ಭಾರತದಲ್ಲಿ ಫ್...",N_NN N_NN N_NNP N_NNP RD_PUNC N_NNP N_NNP PSP ...,O O NEP NEP O NEL NEL O O O
4,5,"ಶ್ರೀ ಜಾವೇದ್ ಅಶ್ರಫ್ , ಫ್ರಾನ್ಸ್ ಮತ್ತು ಮೊನಾಕೊದಲ್ಲ...",N_NN N_NNP N_NNP RD_PUNC N_NNP CC_CCD N_NNP N_...,O NEP NEP O NEL O NEL NEL O O
5,6,"ರಾಯಭಾರಿ ರಾಜೀವ್ ಭಾಟಿಯಾ , ಗಣ್ಯ ವ್ಯಕ್ತಿ , ಗೇಟ್‌ವೇ...",N_NNP N_NNP N_NNP RD_PUNC N_NN N_NN RD_PUNC N_...,O NEP NEP O O O O NEO NEO O
6,7,"ಮಹಿಳೆಯರೇ ಮತ್ತು ಮಹನೀಯರೇ ಹಾಗೂ ವಿಶೇಷ ಭಾಗೀದಾರರೇ ,",N_NN CC_CCD N_NN CC_CCD JJ N_NN RD_PUNC,O O O O O O O
7,8,ನಮಸ್ಕಾರ .,N_NN RD_PUNC,O O
8,9,ಇಂದು ಭಾರತ - ಫ್ರಾನ್ಸ್ ಟ್ರ್ಯಾಕ್ 1.5 ನೇ ಸಂವಾದದಲ್ಲ...,N_NN N_NNP RD_SYM N_NNP N_NNP QT_QTC N_NN N_NN...,O NEL O NEL O NEN O O O O O O O O
9,10,ಜಾಗತಿಕ ಥಿಂಕ್ - ಟ್ಯಾಂಕ್ ಸ್ಪೇಸ್ ನ ಎರಡು ಅತ್ಯಂತ ಸಕ...,JJ N_NN RD_SYM N_NN N_NN PSP QT_QTC RP_INTF JJ...,O O O O O O O O O O O NEO NEO O NEO O O O O O O O


In [3]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-uncased')

In [4]:
label_all_tokens = False
def align_label(texts,labels):
    tokenized_inputs = tokenizer(texts, padding='max_length', max_length=512, truncation=True)
    word_ids = tokenized_inputs.word_ids()
    previous_word_idx = None
    label_ids = []
    for word_idx in word_ids:
        if word_idx is None:
            label_ids.append(-100)
        elif word_idx != previous_word_idx:
            try:
                label_ids.append(labels_to_ids[labels[word_idx]])
            except:
                label_ids.append(-100)
        else:
            try:
                label_ids.append(labels_to_ids[labels[word_idx]] if label_all_tokens else -100)
            except:
                label_ids.append(-100)
    
        previous_word_ids = word_idx
    return label_ids

class DataSequence(torch.utils.data.Dataset):
    def __init__(self, data):
        
        lb = [i.split() for i in data["labels"].values.tolist()]
        txt = data["text"].values.tolist()
        self.texts = [tokenizer(str(i), padding="max_length", max_length=512,
                               truncation=True, return_tensors="pt") for i in txt]
        self.labels = [align_label(i,j) for i,j in zip(txt,lb)]
        
    def __len__(self):
        
        return len(self.labels)
    
    def get_batch_data(self,idx):
        return self.texts[idx]
    
    def get_batch_labels(self, idx):
        return torch.LongTensor(self.labels[idx])
    
    def __getitem__(self, idx):
        batch_data = self.get_batch_data(idx)
        batch_labels = self.get_batch_labels(idx)
        return batch_data , batch_labels
        
            
            

In [5]:
'''text = data['text'].values.tolist()
example = text[10]
l = data['labels'].values.tolist()
label = l[10]
#print sentence
print(example)
tokenized_inputs = tokenizer(example, padding='max_length', max_length=512, truncation=True)
#prints input_ids, token_type_ids, attention_mask
print(tokenized_inputs)
#decode ids back to original sentence
print(tokenizer.decode(tokenized_inputs.input_ids))
#all words of sentence hv the same word_id
word_ids = tokenized_inputs.word_ids()
print(tokenizer.convert_ids_to_tokens(tokenized_inputs["input_ids"]))
print(word_ids)
'''

'text = data[\'text\'].values.tolist()\nexample = text[10]\nl = data[\'labels\'].values.tolist()\nlabel = l[10]\n#print sentence\nprint(example)\ntokenized_inputs = tokenizer(example, padding=\'max_length\', max_length=512, truncation=True)\n#prints input_ids, token_type_ids, attention_mask\nprint(tokenized_inputs)\n#decode ids back to original sentence\nprint(tokenizer.decode(tokenized_inputs.input_ids))\n#all words of sentence hv the same word_id\nword_ids = tokenized_inputs.word_ids()\nprint(tokenizer.convert_ids_to_tokens(tokenized_inputs["input_ids"]))\nprint(word_ids)\n'

In [6]:
#find the unique labels give them an index

labels = [i.split() for i in data['labels'].values.tolist()]
unique_labels = set()

for lb in labels:
    [unique_labels.add(i) for i in lb if i not in unique_labels]
    
labels_to_ids = {k: v for v, k in enumerate(unique_labels)}
ids_to_labels = {v: k for v, k in enumerate(unique_labels)}

data_train,data_val, data_test = np.split(data.sample(frac=1, random_state = 42),
                                         [int(.8*len(data)),int(.9*len(data))])

print(unique_labels)
print(labels_to_ids)

{'NEA', 'NETE', 'O', 'NEO', 'NEN', 'NEP', 'NED', 'NEL'}
{'NEA': 0, 'NETE': 1, 'O': 2, 'NEO': 3, 'NEN': 4, 'NEP': 5, 'NED': 6, 'NEL': 7}


In [7]:
class BertModel(torch.nn.Module):
    def __init__(self):
        super(BertModel, self).__init__()
        self.bert = BertForTokenClassification.from_pretrained('bert-base-multilingual-uncased', num_labels=len(unique_labels))
    
    def forward(self, input_id, mask, label):
        output = self.bert(input_ids=input_id, attention_mask=mask, labels=label, return_dict=False)
        return output
    

In [8]:
def train_loop(model, data_train, data_val):

    train_dataset = DataSequence(data_train)
    val_dataset = DataSequence(data_val)

    train_dataloader = DataLoader(train_dataset, num_workers=4, batch_size=BATCH_SIZE, shuffle=True)
    val_dataloader = DataLoader(val_dataset, num_workers=4, batch_size=BATCH_SIZE)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    optimizer = SGD(model.parameters(), lr=LEARNING_RATE)

    if use_cuda:
        model = model.cuda()

    best_acc = 0
    best_loss = 1000

    for epoch_num in range(EPOCHS):

        total_acc_train = 0
        total_loss_train = 0

        model.train()

        for train_data, train_label in tqdm(train_dataloader):

            train_label = train_label.to(device)
            mask = train_data['attention_mask'].squeeze(1).to(device)
            input_id = train_data['input_ids'].squeeze(1).to(device)

            optimizer.zero_grad()
            loss, logits = model(input_id, mask, train_label)

            for i in range(logits.shape[0]):

              logits_clean = logits[i][train_label[i] != -100]
              label_clean = train_label[i][train_label[i] != -100]

              predictions = logits_clean.argmax(dim=1)
              acc = (predictions == label_clean).float().mean()
              total_acc_train += acc
              total_loss_train += loss.item()

            loss.backward()
            optimizer.step()

        model.eval()

        total_acc_val = 0
        total_loss_val = 0

        for val_data, val_label in val_dataloader:

            val_label = val_label.to(device)
            mask = val_data['attention_mask'].squeeze(1).to(device)
            input_id = val_data['input_ids'].squeeze(1).to(device)

            loss, logits = model(input_id, mask, val_label)

            for i in range(logits.shape[0]):

              logits_clean = logits[i][val_label[i] != -100]
              label_clean = val_label[i][val_label[i] != -100]

              predictions = logits_clean.argmax(dim=1)
              acc = (predictions == label_clean).float().mean()
              total_acc_val += acc
              total_loss_val += loss.item()

        val_accuracy = total_acc_val / len(data_val)
        val_loss = total_loss_val / len(data_val)

        print(
            f'Epochs: {epoch_num + 1} | Loss: {total_loss_train / len(data_train): .3f} | Accuracy: {total_acc_train / len(data_train): .3f} | Val_Loss: {total_loss_val / len(data_val): .3f} | Accuracy: {total_acc_val / len(data_val): .3f}')

LEARNING_RATE = 5e-3
EPOCHS = 5
BATCH_SIZE = 2

model = BertModel()
train_loop(model, data_train, data_val)

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint a

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

100%|██████████| 24/24 [00:59<00:00,  2.77s/it]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|██████████| 24/24 [00:59<00:00,  2.47s/it]


...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

  0%|          | 0/24 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

100%|██████████| 24/24 [01:12<00:00,  3.01s/it]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [9]:
def evaluate(model, data_test):

    test_dataset = DataSequence(data_test)

    test_dataloader = DataLoader(test_dataset, num_workers=4, batch_size=1)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    total_acc_test = 0
    total_loss_test = 0
    
    for test_data, test_label in test_dataloader:

            test_label = test_label.to(device)
            mask = test_data['attention_mask'].squeeze(1).to(device)

            input_id = test_data['input_ids'].squeeze(1).to(device)

            loss, logits = model(input_id, mask, test_label)

            for i in range(logits.shape[0]):

              logits_clean = logits[i][test_label[i] != -100]
              label_clean = test_label[i][test_label[i] != -100]

              predictions = logits_clean.argmax(dim=1)
              acc = (predictions == label_clean).float().mean()
              total_acc_test += acc
              total_loss_test += loss.item()
            

    val_accuracy = total_acc_test / len(data_test)
    print(f'Test Accuracy: {total_acc_test / len(data_test): .3f}')
    print(f'Loss: {total_loss_test / len(data_test): .3f}')


evaluate(model, data_test)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [10]:
def align_word_ids(texts):
  
    tokenized_inputs = tokenizer(texts, padding='max_length', max_length=512, truncation=True)

    word_ids = tokenized_inputs.word_ids()

    previous_word_idx = None
    label_ids = []

    for word_idx in word_ids:

        if word_idx is None:
            label_ids.append(-100)

        elif word_idx != previous_word_idx:
            try:
                label_ids.append(1)
            except:
                label_ids.append(-100)
        else:
            try:
                label_ids.append(1 if label_all_tokens else -100)
            except:
                label_ids.append(-100)
        previous_word_idx = word_idx

    return label_ids


def evaluate_one_text(model, sentence):


    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    text = tokenizer(sentence, padding='max_length', max_length = 512, truncation=True, return_tensors="pt")

    mask = text['attention_mask'].to(device)
    input_id = text['input_ids'].to(device)
    label_ids = torch.Tensor(align_word_ids(sentence)).unsqueeze(0).to(device)

    logits = model(input_id, mask, None)
    logits_clean = logits[0][label_ids != -100]

    predictions = logits_clean.argmax(dim=1).tolist()
    prediction_label = [ids_to_labels[i] for i in predictions]
    print(sentence)
    print(prediction_label)
            




In [11]:
evaluate_one_text(model, 'ಭಾರತವು ಒಂದು ದೇಶವಾಗಿದೆ.')

ಭಾರತವು ಒಂದು ದೇಶವಾಗಿದೆ.
['NEL', 'O', 'O', 'O']
